In [9]:
# Init the Gird components and functions
import numpy as np
from tkinter import W
import heapq


class node():
    def __init__(self,x,y):
        self.x=x
        self.y=y
        self.connections = []
    def connect(self,newnode):
        if(len(self.connections)>0):
            for i in self.connections:
                if(i.x == newnode.x and i.y == newnode.y):return
        
        if((self.x == newnode.x + 1 or self.x == newnode.x - 1) and self.y == newnode.y):
            self.connections.append(newnode)
            newnode.connections.append(self)
            return
        if((self.y == newnode.y + 1 or self.y == newnode.y - 1) and self.x == newnode.x):
            self.connections.append(newnode)
            newnode.connections.append(self)
            return
def randomConnection(grid):
    (w,h) = grid.shape
    x = np.random.randint(0,w)
    y = np.random.randint(0,h)
    neighbor = np.random.randint(0,4)
    if(neighbor==0 and x>0):
        grid[x,y].connect(grid[x-1,y])
        return
    elif(neighbor==1 and x<w-1):
        grid[x,y].connect(grid[x+1,y])
        return
    elif(neighbor==2 and y>0):
        grid[x,y].connect(grid[x,y-1])
        return
    elif(y<h-1):
        grid[x,y].connect(grid[x,y+1])
        return
def fillAll(grid):
    (w,h) = grid.shape
    flag = False
    while(flag==False):
        flag=True
        for i in range(w):
            for j in range(h):
                if(len(grid[i,j].connections)<=0):
                    flag=False
                    neighbor = np.random.randint(0,4)
                    if(neighbor==0 and i>0):
                        grid[i,j].connect(grid[i-1,j])
                    elif(neighbor==1 and i<w-1):
                        grid[i,j].connect(grid[i+1,j])
                    elif(neighbor==2 and j>0):
                        grid[i,j].connect(grid[i,j-1])
                    elif(j<h-1):
                        grid[i,j].connect(grid[i,j+1])




In [10]:
# Init the grid
(w,h) = (50,50)
grid = np.full((h,w),fill_value=node(0,0))
for i in range(h):
    for j in range(w):
        grid[i,j]=node(i,j)

# fillAll(grid=grid)
for i in range(int(w*h*1.6)): randomConnection(grid=grid)
for i in range(int(w*h/20)): randomConnection(grid=grid[0:int(h/3),0:int(w/3)])
for i in range(int(w*h/20)): randomConnection(grid=grid[int(2.0*h/3):h,int(2.0*w/3):w])
fillAll(grid=grid)

matrix = np.zeros((h,w),dtype='uint8')
for i in range(h):
    for j in range(w):
        matrix[i,j]=len(grid[i,j].connections)
# print(matrix)

maze = np.zeros((h*2-1,w*2-1),dtype='uint8')
# maze=maze+1
for i in range(h*2-1):
    for j in range(w*2-1):
        if(i&1==1 or j&1==1):maze[i,j]=1

# print(maze)

In [11]:
# Convert Grid to data of maze
from IPython.display import clear_output
for i in range(h):
    for j in range(w):
        for k in range(len(grid[i,j].connections)):
            if(grid[i,j].x==grid[i,j].connections[k].x):
                if(grid[i,j].y==grid[i,j].connections[k].y+1): maze[i*2,j*2-1]=0
                else: maze[i*2,j*2+1]=0
            else: 
                if(grid[i,j].x==grid[i,j].connections[k].x+1): maze[i*2-1,j*2]=0
                else: maze[i*2+1,j*2]=0
            # clear_output(wait=False)
            # print(maze)

In [12]:
import cv2
truemaze=maze*255
truemaze=255-truemaze
truemaze=cv2.resize(truemaze,(600,600),interpolation=0)
# _,truemaze = cv2.threshold(truemaze,127,255,type=0)

cv2.imshow('maze',truemaze)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
# BFS Search
def stack_BFS(path, root):
    root1=root
    nodestack = []
    while(len(path)>0):
        for node in path.pop():
            for connect in node.connections:
                flag = False
                if(connect.x==root1.x and connect.y==root1.y): 
                    nodestack.append(node)
                    root1=node
                    flag=True
                    break
                if(flag):break
    return nodestack
def BFS_Search(grid,originalmaze):
    (x,y)=grid.shape
    x-=1
    y-=1
    mazecopy=np.zeros((x*2+1,y*2+1),dtype='uint8')
    mazecopy[0,0]=1
    # t=0
    path=[]
    temp=[]
    temp.append(grid[0,0])
    path.append(temp)
    while(True):
        batch=[]
        for node in path[-1]:
            mazecopy[node.x*2,node.y*2]=1
            for connect in node.connections:
                flag = False
                # Check if node is in path
                for n in path:
                    flag1 = False
                    for m in n:
                        if(connect.x==m.x and connect.y==m.y):
                            flag1=True
                            flag=True
                            break
                    if(flag1): break
                mazecopy[node.x*2 + connect.x-node.x,node.y*2 + connect.y-node.y]=1
                if(flag==False): 
                    for n in batch:
                        if(connect.x==n.x and connect.y==n.y):
                            flag=True
                            break

                if(flag): continue
                if(connect.x==x and connect.y==y): 
                    # cv2.waitKey(0)
                    # cv2.destroyAllWindows()
                    # print(t)
                    mazecopy[node.x*2 + connect.x-node.x,node.y*2 + connect.y-node.y]=1
                    mazecopy[connect.x*2,connect.y*2]=1
                    cv2.imshow('maze',originalmaze-cv2.resize(mazecopy*127,(600,600),interpolation=0))
                    return stack_BFS(path,connect)
                
                mazecopy[connect.x*2,connect.y*2]=1
                batch.append(connect)
                # t+=1
                # print(t)
                # cv2.imshow('maze',originalmaze-cv2.resize(mazecopy*127,(600,600),interpolation=0))
        if(len(batch)<=0): 
            # print(t)
            # cv2.destroyAllWindows()
            return
        cv2.imshow('maze',originalmaze-cv2.resize(mazecopy*127,(600,600),interpolation=0))
        path.append(batch)
        cv2.waitKey(10)
        

In [14]:
# DFS Search
def DFS_Search(grid, originalmaze):
    (x,y)=grid.shape
    x-=1
    y-=1
    path=[[grid[0,0],0]]
    root=grid[0,0]
    trash=[]
    mazecopy=np.zeros((x*2+1,y*2+1),dtype='uint8')
    mazecopy[0,0]=1
    while(True):
        if(root.x==x and root.y==y): return path

        
        turn=0
        while(turn<len(root.connections)):
            flag=True
            for i in reversed(path):
                if(root.connections[turn].x==i[0].x and root.connections[turn].y == i[0].y):
                    turn+=1
                    flag=False
                    break
            if(flag):
                for j in trash:
                    if(root.connections[turn].x==j.x and root.connections[turn].y == j.y):
                        turn+=1
                        flag=False
                        break
            if(flag): break
        if(turn>=len(root.connections)):
            trash.append(path.pop()[0])
            try:
                root=path[-1][0]
            except: return
            continue


        root=path[-1][0].connections[turn]
        mazecopy[root.x*2,root.y*2]=1
        mazecopy[path[-1][0].x*2 + root.x - path[-1][0].x, path[-1][0].y*2+root.y-path[-1][0].y]=1
        cv2.waitKey(1)
        cv2.imshow('maze',originalmaze-cv2.resize(mazecopy*127,(600,600),interpolation=0))
        path.append([root,0])
        
        
    pass

In [15]:
#A Star
class Node:
    def __init__(self, x, y, h_cost):
        self.x = x
        self.y = y
        self.h_cost = h_cost
        self.g_cost = float('inf')
        self.f_cost = float('inf')
        self.parent = None

    def __lt__(self, other):
        return self.f_cost < other.f_cost

def calculate_heuristic(node, goal):
    return abs(node.x - goal.x) + abs(node.y - goal.y)

def reconstruct_path(node):
    path = []
    current = node
    while current is not None:
        path.append(current)
        current = current.parent
    return path[::-1]

def AStar_Search(grid, originalmaze):
    (x, y) = grid.shape
    x -= 1
    y -= 1
    mazecopy = np.zeros((x * 2 + 1, y * 2 + 1), dtype='uint8')
    mazecopy[0, 0] = 1

    start = Node(0, 0, 0)
    goal = Node(x, y, 0)

    open_set = []
    closed_set = set()

    start.g_cost = 0
    start.f_cost = start.g_cost + calculate_heuristic(start, goal)

    heapq.heappush(open_set, start)

    while open_set:
        current = heapq.heappop(open_set)
        closed_set.add((current.x, current.y))

        if current.x == goal.x and current.y == goal.y:
            mazecopy[current.x * 2, current.y * 2] = 1
            cv2.imshow('maze', originalmaze - cv2.resize(mazecopy * 127, (600, 600), interpolation=0))
            path = reconstruct_path(current)
            return path

        mazecopy[current.x * 2, current.y * 2] = 1

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x = current.x + dx
            new_y = current.y + dy

            if new_x < 0 or new_x > x or new_y < 0 or new_y > y:
                continue

            if grid[new_x, new_y] == 0 or (new_x, new_y) in closed_set:
                continue

            neighbor = Node(new_x, new_y, calculate_heuristic(Node(new_x, new_y, 0), goal))
            tentative_g_cost = current.g_cost + 1

            if tentative_g_cost < neighbor.g_cost:
                neighbor.parent = current
                neighbor.g_cost = tentative_g_cost
                neighbor.f_cost = neighbor.g_cost + neighbor.h_cost

                if (new_x, new_y) not in closed_set:
                    heapq.heappush(open_set, neighbor)

        cv2.imshow('maze', originalmaze - cv2.resize(mazecopy * 127, (600, 600), interpolation=0))
        cv2.waitKey(1)
    
    return []
# Usage:
decoy = grid
cv2.imshow('maze', truemaze)
k = AStar_Search(grid=decoy, originalmaze=truemaze)
cv2.waitKey(0)
cv2.destroyAllWindows()
# print(len(k))

In [16]:
#Uniform Cost Search
class UCSNode:
    def __init__(self, state, cost, parent=None):
        self.state = state
        self.cost = cost
        self.parent = parent

    def __lt__(self, other):
        return self.cost < other.cost

def ucs_search(grid, original_maze):
    (x, y) = grid.shape
    x -= 1
    y -= 1
    maze_copy = np.zeros((x * 2 + 1, y * 2 + 1), dtype='uint8')
    maze_copy[0, 0] = 1

    start_node = UCSNode(state=grid[0, 0], cost=0)
    priority_queue = [start_node]
    visited = set()

    while priority_queue:
        current_node = heapq.heappop(priority_queue)

        if current_node.state.x == x and current_node.state.y == y:
            return reconstruct_path(current_node)

        if current_node.state in visited:
            continue

        visited.add(current_node.state)

        for connect in current_node.state.connections:
            new_cost = current_node.cost + 1
            new_node = UCSNode(state=connect, cost=new_cost, parent=current_node)
            heapq.heappush(priority_queue, new_node)

            maze_copy[current_node.state.x * 2 + connect.x - current_node.state.x,
                      current_node.state.y * 2 + connect.y - current_node.state.y] = 1

            maze_copy[connect.x * 2, connect.y * 2] = 1

            cv2.imshow('maze', original_maze - cv2.resize(maze_copy * 127, (600, 600), interpolation=0))
            cv2.waitKey(1)

    return None

def reconstruct_path(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]

In [17]:
decoy = grid
cv2.imshow('maze', truemaze)
ucspath = ucs_search(grid=decoy, original_maze=truemaze)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(len(ucspath))

# while ucspath:
#   temp = ucspath.pop()
#   print(temp.x, temp.y)


99


In [18]:
cv2.imshow('maze',truemaze)
k = DFS_Search(grid,truemaze)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(len(k))
# for n in k:
#     print(n[0].x,n[0].y)

397


In [19]:
decoy = grid
cv2.imshow('maze',truemaze)
k = BFS_Search(grid=decoy,originalmaze = truemaze)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(len(k))
# while(len(k)>0):
#     temp = k.pop()
#     print(temp.x,temp.y)

98
